# Anecdotes



## Import Data

In [1]:
import datetime
import os

import fiona
import geopandas as gpd
import geopy.distance
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
from loguru import logger
fiona.drvsupport.supported_drivers["kml"] = "rw"
fiona.drvsupport.supported_drivers["KML"] = "rw"
storage = "/Volumes/easystore/Drones"
coi = [
    "Incident No.",
    "Description",
    "cfs-date",
    "Block Location",
    "Latitude",
    "Longitude",
    "drone-id",
]

calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

flight_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/mapping/cv-blocks.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks["GEOID20"] = cv_blocks["GEOID20"].apply(lambda x: x[3:])
cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

## Norell Martinez

Norell Martinez, a 60-year-old professor of English and Chicana/o Studies, has lived on the West Side of Chula Vista her entire life. A first-generation immigrant, her parents migrated from Tijuana to Chula Vista when she was one year old. “A lot of people on the West side share a similar background as me; it’s a diverse community,” Martinez says. 

Martinez lives a block and a half from police headquarters on a street that is among the safest on the West side. In the last two years, however, drones have flown by at least 500 times, in the process amassing nearly five hours of footage from the sky above her block.

Before the drone program started, she says, her neighborhood was quiet. Now the sound of the rotors keeps her up at night. “We pay a lot of money and make a lot of sacrifices to have a little tiny piece of property that’s ours,” she says. “It feels like our home is not ours anymore. It’s like it belongs to the Chula Vista Police Department.”

463 G Street


In [4]:
nm_fb = flight_blocks[flight_blocks["GEOID20"] == "060730127003001"]

In [8]:
logger.info(f"There have been {nm_fb['id'].drop_duplicates().shape[0]} flights over Martinez' house since July 2021")

2024-05-24 11:38:18.384 | INFO     | __main__:<module>:1 - There have been 959 flights over Martinez' house since July 2021


In [9]:
logger.info(f"These flights have amassed {nm_fb['seconds'].astype(int).sum()/3600} hours of footage over her house since July 2021")

2024-05-24 11:38:18.514 | INFO     | __main__:<module>:1 - These flights have amassed 4.754444444444444 hours of footage over her house since July 2021


In [10]:
# 060730131032011 <- harborside
# 060730125023005 <- circlek

In [11]:
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])
cv_flights["drone-date"] = pd.to_datetime(cv_flights["date"])

## Vistan
Few places in Chula Vista have had drones dispatched to them more frequently than the Vistan Apartments on the West side. The complex, in one of the poorest neighborhoods in Chula Vista, is populated largely by Black and Latine families, many of whom are immigrants. Over the last two years, CVPD drones have spent a total of TK hours flying above the Vistan Apartments and have been dispatched to respond to incidents ranging from indecent exposure to even an attempted kidnapping. 

Vistan Apartments resident Jesus Lopez says that one afternoon, while he was away, a man banged loudly on his front door. When his three-year-old opened it, the stranger reached in and tried to grab his kid. According to Lopez, his babysitter quickly yanked the child inside, slammed the door, and dialed 911. Within minutes, Lopez says, a police drone arrived to search for the suspect. 

The department did not find the individual, but, nevertheless, Lopez—who spoke to WIRED through his locked door, because he doesn't open it for people he doesn’t recognize anymore—says that police drones make him feel safer. 

In [12]:
vistan_flights = flight_blocks[(flight_blocks['GEOID20']=='060730125012001')]

logger.info(f"{vistan_flights['id'].drop_duplicates().shape[0]} flights passed through vistan")
logger.info(f"{vistan_flights['seconds'].astype(int).sum()/3600} hours of footage")

2024-05-24 11:38:19.098 | INFO     | __main__:<module>:3 - 312 flights passed through vistan
2024-05-24 11:38:19.100 | INFO     | __main__:<module>:4 - 8.018888888888888 hours of footage


In [13]:
vistan = cv_flights[cv_flights['id'].isin(vistan_flights['id'])]
vistan[vistan['address_map'].str.contains('oaklawn',case=False)]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,clean-type,match,drone-date
487,32.6332605,-117.0939602,4:39pm,358959ed97cdcfe8958ddfa58c1e7968,2021-07-26,4:39pm,400 Oaklawn Ave.,CVL62166,Welfare Check,cvpd,welfare check,62166,2021-07-26
1067,32.6332605,-117.0939602,3:08pm,506d4bb6d4b80661b1025d9d05d575bc,2021-08-24,3:08pm,400 Oaklawn Ave.,CVL70924,Domestic Violence,cvpd,domestic violence,70924,2021-08-24
1171,32.6322368,-117.0930146,5:15pm,3ec63a4e3fb70e9067a231929ba2ca1b,2021-08-30,5:15pm,400 Oaklawn Ave.,CVL72781,Subject Causing Disturbance,cvpd,subject causing disturbance,72781,2021-08-30
1420,32.6332605,-117.0939602,12:52pm,eae341652fc042d7784043e106ba8047,2021-09-14,12:52pm,400 Oaklawn Ave.,CVL77382,Vandalism,cvpd,vandalism,77382,2021-09-14
1639,32.6300709,-117.0926315,10:47am,25acc9c481fce6187b6142ab5a9db554,2021-09-27,10:47am,H St./ Oaklawn Ave.,CVL81243,Weapon Threat,cvpd,weapon threat,81243,2021-09-27
1901,32.630112,-117.092491,7:05am,49e0dd2f9bffad3a67557bca8930793a,2022-01-16,7:05am,H St/Oaklawn Ave,CV2200647,Robbery,cvpd,robbery,00647,2022-01-16
2047,32.6332605,-117.0939602,10:16am,88a82a56bca56ec17a73ebe6b0ce6af6,2022-01-26,10:16am,400 Oaklawn Ave.,CVL7050,Domestic Violence,cvpd,domestic violence,07050,2022-01-26
2065,32.6128055,-117.0857634,7:24am,917e00a291fd5e561b7e1f0c70a63754,2022-01-27,7:24am,1000 Oaklawn Ave.,CVL7333,Vehicle Theft,cvpd,vehicle theft,07333,2022-01-27
2165,32.6332605,-117.0939602,11:24am,b6dd6d612499fc3e02e78cf7dce3b926,2022-02-02,11:24am,400 Oaklawn Ave.,CVL9221,Subject Causing Disturbance,cvpd,subject causing disturbance,09221,2022-02-02
2308,32.6332598,-117.0939625,11:21am,a2b86a82a8a4143df8339238fefb21ce,2022-02-11,11:21am,400 Oaklawn,L012106,Man with a Gun,cvpd,man with a gun,12106,2022-02-11


,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,clean-type,match
0,32.63998083,-117.08271223,1:55pm,55a1fb6dab54e24b8537c7f12a4fe725,2021-07-01,1:55pm,315 4th Ave,NaN,NaN,cvpd,NaN,NaN
1,32.61408,-116.975275,4:20pm,15759cef724c96374c1a93419d41dd53,2021-07-01,4:20pm,WOLF CANYON/PEABODY,L054372,DOMESTIC VIOLENCE,cvpd,domestic violence,54372
2,32.630147,-117.094228,4:09pm,b7235cf661360dc6e3ad6dad7aff335f,2021-07-01,4:09pm,745 H,L054334,UNKNOWN PROBLEM,cvpd,unknown problem,54334
3,32.6483145,-117.092483,4:02pm,171d95280b64377eafadaef80590acd0,2021-07-01,4:02pm,500 SEA VALE CT,L054331,TRAFFIC COLLISION,cvpd,traffic collision,54331
4,32.6175204,-117.0713635,3:05pm,2dde46cad7d7d0103a9b42dcf9514aec,2021-07-01,3:05pm,330 MOSS,L054364,DOMESTIC VIOLENCE,cvpd,domestic violence,54364
...,...,...,...,...,...,...,...,...,...,...,...,...
9554,32.5989197,-117.0615246,2:14pm,e4fdf51abb9f2d999af03177b0e2595d,2023-09-02,2:14pm,275 Orange Ave.,CVL75348,Domestic Violence,cvpd,domestic violence,75348
9555,32.6098649,-117.0813317,12:55pm,32c3272b9e976103882e5fe03b020112,2023-09-02,12:55pm,Broadway/ Naples St.,CVL75330,Traffic Hazard,cvpd,traffic hazard,75330
9556,32.6495326,-117.0285142,2:04am,5365db931f56c1c04dea07c19813dc7e,2023-09-02,2:04am,400 Milagrosa Circle,L075219,Burglary,cvpd,burglary,75219
9557,32.59246813605942,-117.0126979673828,6:40pm,136cbafe6c19237c10fd27fd99fb75ad,2023-09-02,6:40pm,800 Main St.,CVL75399,Duress Alarm,cvpd,duress alarm,75399
